In [8]:
import tensorflow as tf
print("Tensorflow version =",tf.__version__)
import numpy as np
import os
import time
import random
import argparse
import itertools
import ComplexRNNwavefunction
from ComplexRNNwavefunction import RNNwavefunction
import gc
from tensorflow.python.client import device_lib
import sys
sys.argv = [sys.argv[0]]
def get_numavailable_gpus():
    local_device_protos = device_lib.list_local_devices()
    print(local_device_protos)
    return len([x.name for x in local_device_protos if x.device_type == 'GPU'])

parser = argparse.ArgumentParser()
parser.add_argument('--system_size', type = int, default=4)
parser.add_argument('--numsamples', type = float, default=256)
parser.add_argument('--rnn_unit', type = float, default=16)
parser.add_argument('--num_unit', type = int, default=128)
parser.add_argument('--num_first_sample', type = int, default=100)
parser.add_argument('--num_second_sample', type = int, default=1500)
parser.add_argument('--position_', type = int, default=11)
parser.add_argument('--param', type=int, default = 1)
args = parser.parse_args()



systemsize = args.system_size
num_unit = args.num_unit
position_ = args.position_
numsamples = args.numsamples
Marshall_sign = True
rnn_unit = args.rnn_unit
num_layers = 1

learningrate = 0.01

seed = args.param
N = systemsize #Number of spins
lr = np.float64(learningrate)

units = [rnn_unit]

#Seeding
tf.reset_default_graph()
random.seed(seed)  # `python` built-in pseudo-random generator
np.random.seed(seed)  # numpy pseudo-random generator
tf.set_random_seed(seed)  # tensorflow pseudo-random generator


path=os.getcwd()

ending='_units'
for u in units:
    ending+='_{0}'.format(u)

savename = '_ES'

filename='../Check_Points/ES/RNNwavefunction_N'+str(N)+'_samp'+str(numsamples)+'_lradap'+str(lr)+'_complexGRURNN'+ savename + ending +'_zeromag.ckpt'

'''
ckpt_meta_path = filename+".ckpt"
print(os.path.exists(ckpt_meta_path))
'''


# Intitializing the RNN-----------

input_dim=2 #Dimension of the Hilbert space for each site (here = 2, up or down)
numsamples_=20 #only for initialization; later I'll use a much larger value (see below)


wf=RNNwavefunction(N,units=units,cell=tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell, seed = seed) #contains the graph with the RNNs


'''
 #contains the graph with the RNNs
sampling=wf.sample(numsamples_,input_dim) #call this function once to create the dense layers
'''
with tf.variable_scope(wf.scope,reuse=tf.AUTO_REUSE):
    with wf.graph.as_default():
        if (get_numavailable_gpus()==0):
            device = '/CPU:0'
        else:
            device = '/GPU:0'
        
        global_step = tf.Variable(0, trainable=False)
        learningrate_placeholder=tf.placeholder(dtype=tf.float32,shape=[])
        learningrate_withexpdecay = tf.train.exponential_decay(learningrate_placeholder, global_step, decay_steps = 100, decay_rate = 1.0, staircase=True) #Adaptive Learning (decay_rate = 1 -> no decay)

        inputs=tf.placeholder(dtype=tf.int32,shape=[None,N]) #the inputs are the samples of all of the spins

        with tf.device(device):
                        
            sample_first_input = tf.stop_gradient(wf.sample(num_unit, inputdim=2))
            log_amplitudes_=tf.stop_gradient(wf.log_amplitude(inputs,inputdim=2))
            cond_prob = tf.stop_gradient(wf.cond_prob(inputs, inputdim =2))
            
            #cost = 2*tf.real(tf.reduce_mean(tf.conj(log_amplitudes_)*tf.stop_gradient(Eloc)) - tf.conj(tf.reduce_mean(log_amplitudes_))*tf.reduce_mean(tf.stop_gradient(Eloc)))
        optimizer=tf.train.AdamOptimizer(learning_rate=learningrate_withexpdecay, beta1=0.9, beta2 = 0.999, epsilon = 1e-8)
        #gradients, variables = zip(*optimizer.compute_gradients(cost))
        #clipped_gradients = [tf.clip_by_value(g, -10, 10) for g in gradients]
        #optstep=optimizer.apply_gradients(zip(clipped_gradients,variables),global_step=global_step)
        init=tf.global_variables_initializer()
        saver=tf.train.Saver()

# End Intitializing

#Starting Session------------
#Activating GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.Session(graph=wf.graph, config=config)
sess.run(init)
#---------------------------

         #define tf saver
with wf.graph.as_default():
    total_parameters = 0
    for variable in tf.compat.v1.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(shape)
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim
        print(variable_parameters)
        total_parameters += variable_parameters
    print(total_parameters)

with tf.compat.v1.variable_scope(wf.scope,reuse=tf.compat.v1.AUTO_REUSE):
    with wf.graph.as_default():
        try:
            print("Loading the model from checkpoint")
            saver.restore(sess,filename)
            print("Loading was successful!")
        except:
            print("Loading was failed!")
            print(filename)
            sys.exit()

Tensorflow version = 1.13.1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2995630230043900288
]
one_hot_samples: Tensor("one_hot_1:0", shape=(?, 4, 2), dtype=float32, device=/device:CPU:0)
cond_prob: Tensor("Sum_2:0", shape=(?, 4), dtype=float32, device=/device:CPU:0)
(18, 32)
576
(32,)
32
(2, 16)
32
(16, 16)
256
(16,)
16
(16,)
16
(16, 2)
32
(2,)
2
(16, 2)
32
(2,)
2
996
Loading the model from checkpoint
INFO:tensorflow:Restoring parameters from ../Check_Points/ES/RNNwavefunction_N4_samp256_lradap0.01_complexGRURNN_ES_units_16_zeromag.ckpt


INFO:tensorflow:Restoring parameters from ../Check_Points/ES/RNNwavefunction_N4_samp256_lradap0.01_complexGRURNN_ES_units_16_zeromag.ckpt


Loading was successful!


In [9]:
with tf.compat.v1.variable_scope(wf.scope,reuse=tf.compat.v1.AUTO_REUSE):
    with wf.graph.as_default():
        basis = np.array(list(itertools.product([0, 1], repeat=N)))
        log_amp_all_basis = sess.run(log_amplitudes_, feed_dict={inputs: basis})
    

In [13]:
prob_all_basis=(np.abs(np.exp(log_amp_all_basis))**2)
print(prob_all_basis)
amp = (np.exp(log_amp_all_basis))
print(amp)

[0.         0.         0.         0.12955439 0.         0.12102269
 0.24142122 0.         0.         0.25908306 0.12930974 0.
 0.11960894 0.         0.         0.        ]
[-0.        +0.j         -0.        +0.j         -0.        +0.j
 -0.24779299-0.26106137j -0.        -0.j         -0.0782396 -0.3389709j
 -0.11050337-0.47875905j -0.        +0.j         -0.        +0.j
 -0.3424361 -0.37659073j -0.24049339-0.26734373j -0.        -0.j
  0.0724014 +0.33818185j -0.        +0.j          0.        +0.j
  0.        +0.j        ]


In [24]:
np.load("tv_first_var0.0.npy")

array([[0.05209623, 0.05097875, 0.05574478, 0.0429248 ]])

In [11]:
np.load("tv_second_var0.24.npy")

array([[0.10667124, 0.02259604, 0.04029338, 0.017396  , 0.02504426]])

In [62]:
np.load("../Check_points/J1J2/meanEnergy_N64_samp128_lradap0.0005_complexGRURNN_J1J20.25_units_64_zeromag.npy")

array([ 19.602417-0.02014877j,  19.524143+0.05143145j,
        19.558655-0.0539176j , ..., -25.553974+0.02380841j,
       -25.551773-0.00339204j, -25.600811+0.00493468j], dtype=complex64)

In [71]:
np.where(a==-1)[0]

array([3], dtype=int64)

In [51]:
position = 4
L = 10
step = 2
label = np.zeros((L))
for i in range (position+1):   #position follows the label in python tradition
    label[i] = np.abs(position-i)
label[position] = -1
length = np.max(label)
output = 0
# get the coordinate that we need to match
for i in range (1, step+1):
    pos_temp = np.where(label == i)
    label[pos_temp] = -1

label_where = np.where(label==-1)
print(label_where)
print(label)

(array([2, 3, 4], dtype=int64),)
[ 4.  3. -1. -1. -1.  0.  0.  0.  0.  0.]


In [52]:
label[np.where(label_where==np.array(2))[0]]

array([4.])

In [41]:
from ComplexRNNwavefunction import RNNwavefunction

systemsize = 64
J1_  = 1.0
J2_ = 0.24
Marshall_sign = True
num_units = 64
num_layers = 1
numsamples = 128
learningrate = 5*1e-4
seed = 111
N=systemsize #Number of spins
lr = np.float64(learningrate)

J1=+J1_*np.ones(N) # nearest neighbours couplings
J2=+J2_*np.ones(N) # next-nearest neighbours couplings

units = [num_units]

#Seeding
tf.compat.v1.reset_default_graph()
np.random.seed(seed)  # numpy pseudo-random generator

path = "/../Check_Points/J1J2/RNNwavefunction_N64_samp128_lradap0.0005_complexGRURNN_J1J20.24_units_64_zeromag.ckpt"

wf=RNNwavefunction(N,units=units,cell=tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell, seed = seed) # tensorflow pseudo-random generator
 
with tf.compat.v1.variable_scope(wf.scope,reuse=tf.compat.v1.AUTO_REUSE):
    with wf.graph.as_default():
        saver=tf.compat.v1.train.Saver()

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [55]:
len([[4,6]])

1

In [14]:
np.array(2)

array(2)

In [17]:
np.where(np.array([1,2,3])==np.array(2))[0]

array([1], dtype=int64)

In [4]:
import numpy as np
a = np.ones(5)

In [6]:
print(slice(0, 5))

slice(0, 5, None)
